# Visualização de dados: Trabalho 2

Trabalho da disciplina de Visualização de Dados da Universidade Federal Fluminense, do período de 2023.2.
O intuito desse trabalho é atráves de um dataset de nossa escolha, definir perguntas e mostrar discussões associadas as melhores ou mais adequadas visualizações.

Cada pergunta deve conter as seguintes partes:
 * Uma visualização (construída utilizando a biblioteca vega-lite);
 * Uma discussão sobre as razões que motivaram a escolha da visualização desenvolvida;
 * Uma análise das informações que podem ser obtidas com a interepretação da visualização.

Não existe número definido de quantas perguntas devo responder, porém devo definir depois de fazer uma análise exploratória do dados.

Além disso, espera-se que as análises gerem insights interessantes sobre os datasets escolhidos. Portanto, tentarei desenvolver as análises além do básico, para que conclusões e resultados surpreendentes sejam obtidos.

Com o objetivo de deixar formatado e mais bem comentado, para maior compreensão do que estou fazendo, usarei o ChatGPT para esse artifício e me ajudar com dúvidas de programação.

### Fonte de dados

Os dados foram retirados do site do [data.gov](https://data.gov/), site de dados abertos dos Estados Unidos da América, que dá acesso a diversos datasets publicados por agências do governo federal americano. Mais especificamente, os dados estão associados aos crimes que aconteceram em Los Angeles, de 2020 para os dias atuais. [Link aqui](https://catalog.data.gov/dataset/crime-data-from-2020-to-present).

Os dados foram transcritos de relatórios de crimes no papel, e por isso pode haver uma certa inacurácias em relação aos dados. Há alguns campos com dados faltantes e os campos de endereços são só colocados as quadras mais próximas para manter a privacidade.


---

**Roteiro**:
1. Análise Exploratória dos Dados e tratamento do Dataset;
2. Definição de Perguntas;
3. Respostas das Perguntas no formato definido;


Começando com a importação das bibliotecas e dos dados:

In [1]:
import pandas as pd
import altair as alt
from datetime import datetime

In [2]:
df_original = pd.read_csv('../dados/Crime_Data_from_2020_to_Present.csv')

Para facilitar os entendimentos das colunas, segue o [link](https://data.lacity.org/Public-Safety/Crime-Data-from-2020-to-Present/2nrs-mtv8) com algumas explicações.

In [3]:
# df_original.info()

In [4]:
pd.set_option('display.max_columns', None)
# df_original.head()

Existe algumas colunas que estão codificadas de acordo com o crime e com a arma utilizada, então não temos como trabalhar com essas colunas, Podemos trabalhar com suas descrições. Devemos tirá-las para não ficar carregando colunas extras para o problema.

Colunas que podemos tirar que estão codificadas:
* DR_NO
* Tudo de que tive 'Cd' e somente 'Status', com exceção das descrições ('Desc')


---
Além disso, não vale a pena alterar muitos valores strings, pois como vieram de relatórios escritos, por mais que possamos agrupar, acredito que o trabalho manual e a incerteza de categorização pode ser informação perdida.

## Tratamento de Dados e EDA

In [5]:
lista_exclusao = []
for col in df_original.columns:
    if 'Cd' in col:
        lista_exclusao.append(col)
# Tirar caso de descrição de crime
lista_exclusao.pop(lista_exclusao.index('Crm Cd Desc'))
# Adicionando casos extras que não são úteis
lista_exclusao.extend(['Status', 'Rpt Dist No', 'DR_NO', 'AREA'])

In [6]:
# Novo dataframe que será trabalhado
df_mod = df_original.drop(lista_exclusao, axis=1)

In [7]:
# Tirando codificação
decode = {
    'A' : 'Other Asian',
    'B' : 'Black',
    'C' : 'Chinese',
    'D' : 'Cambodian',
    'F' : 'Filipino',
    'G' : 'Guamanian',
    'H' : 'Hispanic/Latin/Mexican',
    'I' : 'American Indian/Alaskan Native',
    'J' : 'Japanese',
    'K' : 'Korean',
    'L' : 'Laotian',
    'O' : 'Other',
    'P' : 'Pacific Islander',
    'S' : 'Samoan',
    'U' : 'Hawaiian',
    'V' : 'Vietnamese',
    'W' : 'White',
    'X' : 'Unknown',
    'Z' : 'Asian Indian'
}

df_mod['Vict Descent'] = df_mod['Vict Descent'].replace(decode)

# Alteração de tipos de colunas
df_mod['Date Rptd'] = pd.to_datetime(df_mod['Date Rptd'], format='%m/%d/%Y %I:%M:%S %p')
df_mod['DATE OCC'] = pd.to_datetime(df_mod['DATE OCC'], format='%m/%d/%Y %I:%M:%S %p')
df_mod['TIME OCC'] = df_mod['TIME OCC'].apply(lambda x: datetime.strptime(str(x).zfill(4), "%H%M")).dt.time

# criação de novas colunas
df_mod['Date diff Rptd - occ'] = df_mod['Date Rptd'] - df_mod['DATE OCC']

In [8]:
df_mod.describe(include=['O'])

,TIME OCC,AREA NAME,Crm Cd Desc,Mocodes,Vict Sex,Vict Descent,Premis Desc,Weapon Desc,Status Desc,LOCATION,Cross Street
count,838901,838901,838901,722647,728346,728338,838395,292391,838901,838901,133812
unique,1439,21,138,279050,5,20,306,79,6,64053,9781
top,12:00:00,Central,VEHICLE - STOLEN,0344,M,Hispanic/Latin/Mexican,STREET,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",Invest Cont,800 N ALAMEDA ST,BROADWAY
freq,29807,56587,89929,34189,345942,257152,212428,156686,671291,1527,2218


_Análise sobre esses dados categóricos_

In [9]:
df_mod['Vict Sex'].value_counts()

Vict Sex
M    345942
F    308734
X     73578
H        91
-         1
Name: count, dtype: int64

_não sei se é necessário trabalhar em cima desse campo, não sei se é necessário._

In [10]:
# Quantidade de crimes reportados no mesmo dia
perc = df_mod['Date diff Rptd - occ'].value_counts().head(1).sum()/df_mod.shape[0]
print('Porcentagem de Ocorrências feitas no mesmo dia que o crime: {:.2f}%'.format(perc*100))

Porcentagem de Ocorrências feitas no mesmo dia que o crime: 49.35%


In [11]:
# Quantidade de crimes reportado dentro de uma semana
perc = df_mod['Date diff Rptd - occ'].value_counts().head(8).sum()/df_mod.shape[0]
print('Porcentagem de Ocorrências feitas na mesma semana que o crime: {:.2f}%'.format(perc*100))

Porcentagem de Ocorrências feitas na mesma semana que o crime: 87.34%


_Leve análise sobre Ocorrências_

## Definição de Perguntas


De onde eu quero me mover, ou pelo menos penso, não necessariamente vou conseguir fazer todas elas, mas a ideia é que eu produza diversos gráficos sobre um mesmo tópico e vá tentando melhorar os insights:
 * Quantidade de crime por uma média, que ela é móvel e eu vou verificando como ela vai se comportando com o passar do tempo ou pegar médias em tempos definidos, semanas ou meses pelo ano -> Penso em aplicar duas ideias a priori, e ver qual que vale mais a pena. 
   1. A primeira seria, pelo menos gostaria de fazer um circulo temporal (tipo o do @ed_hawkins) que define a quantidade de crime ocorridos durante o dia e como essa média vai crescendo. se possível mensal. Ou seja, o círculo seria dividido em 24 partes (horas) e a linha que vai rotacionando seria a representação dos meses e como estão as quantidades de crimes por mês. Não sei se é possível, mas uma ideia alternativa é um gráfico de linha, com eixo horizontal sendo as horas do dia e vertical seria as quantidades de crimes, e múltiplas linhas representando anos. Posso criar o Filtro para definir as áreas onde aconteceram.
   2. A segunda ideia é fazer um calendário explicito anual (cada linha é um mês), e tentar enxergar algum insight diferenciado, basicamente um heat map de acordo com o ano. (Como são 3 anos, talvez poderia fazer diferente. Cada bloco é uma semana - 52 blocos por ano.) Posso fazer cada ano separado, eu acho interessante essa ideia, é um outro lado da ideia de crimes por hora, aqui eu estaria somente vendo a quantidade por dia.
   3. A terceira ideia é a mais padrão que envolve um gráfico de linha, temporal que mostra o comportamento com o passar dos anos. Esse daqui não seria por hora, mas um formato cumulativo por dia mesmo, de forma total.

   
Existe outras duas ideias que estão mais embrionárias, que são:
 * Dada uma região de los angeles, existe alguma diferença entre os tipos de crime que são cometidos? Essa diferença pode se envolvida por arma utilizada, crime cometido e localidade(onde ocorreu de fato o crime); e podemos também juntar o status desse report gerado. 
 
 Essa ideia pode ser resolvida de algumas maneiras:
 
    1. Um gráfico geoespacial que pode focar as regiões e de acordo com o agrupamento feito, a gente pode ver qual área tem o principal tipo de crime ou característica de nossa escolha. Ou principais
    2. Um gráfico de barras que mostra por região como cada uma delas tem uns principais características.

Para ambas as ideias eu acho que uma barra de linha temporal no qual podemos filtrar é interessante pois podemos escolher em qual época podemos nos focar. Fazer junto o de região e por barras, e quando filtrar na região irá modificar a de barras!


Por fim eu poderia fazer alguma análise em cima das vítimas, e como são as principais vítimas por região e por época, porém tem muito valor nulo, quase um oitavo dos valores. Vamos vendo o que dá para fazer.

---

### Quantidade de crimes por um determinado tempo

A primeira ideia em relação a isso seria da utilização de horas do dia para definir quando tem acontecido crimes. Acho que podemos agrupas por anos, mas antes fazer uma categoria para definir em que hora do dia ocorreu.

In [12]:
# criando cópia para não fazer modificações desnecessárias na df_mod
q1_var1 = df_mod.copy()

In [13]:
# Categorizando em 24 categorias o tempo
q1_var1['TIME HOUR'] = q1_var1['TIME OCC'].apply(lambda x: str(x)[:2])

In [14]:
q1_var1_1 = q1_var1.groupby([pd.Grouper(key='DATE OCC', freq='1Y'), 'TIME HOUR', 'AREA NAME'])[['Date Rptd']].count()
q1_var1_1

Date Rptd
DATE OCC   TIME HOUR AREA NAME             
2020-12-31 00        77th Street        585
                     Central            448
                     Devonshire         281
                     Foothill           230
                     Harbor             451
...                                     ...
2023-12-31 23        Topanga            275
                     Van Nuys           307
                     West LA            282
                     West Valley        309
                     Wilshire           385

[2016 rows x 1 columns]

Ótimo! Tenho o valor de quantidade de crimes por ano, hora e por local. Com isso eu posso fazer a separação para criar o gráfico principal. 

In [15]:
q1_var1_1.reset_index(inplace=True)
q1_var1_1['DATE OCC'] = q1_var1_1['DATE OCC'].dt.year

In [16]:
renomear = {
    'DATE OCC': 'Ano',
    'TIME HOUR': 'Hora',
    'AREA NAME': 'Nome da Área',
    'Date Rptd': 'Quantidade de Crimes'
}

q1_var1_1.rename(columns = renomear, inplace=True)

Formatado do dataframe, Vamos a visualização:

_Motivação porque eu escolhi dessa forma:_

In [37]:
# Filtra por cidade
options = q1_var1_1['Nome da Área'].unique().tolist()
labels = [option + ' ' for option in options]

input_dropdown = alt.binding_radio(
    # Add the empty selection which shows all when clicked
    options=options + [None],
    labels=labels + ['All'],
    name='Áreas: '
)
selection = alt.selection_point(
    fields=['Nome da Área'],
    bind=input_dropdown,
)


alt.Chart(q1_var1_1).mark_line(interpolate='natural', point=True).encode(
    # Eixo X: Quantidade de jogos
    x=alt.X('Hora',
        type='ordinal',
        title='Hora do dia (Formato 24 horas)',
        axis=alt.Axis(
            titleFontSize=16,  # Tamanho da fonte do título
            labelAngle=-30,     # Ângulo dos rótulos do eixo X
            labelFontSize=13   # Tamanho da fonte dos rótulos do eixo X
        )
    ),
    y=alt.Y('sum(Quantidade de Crimes)',
        type='quantitative',
        title='Crimes Reportados',
        axis=alt.Axis(
            titleFontSize=16,  # Tamanho da fonte do título
            tickCount=4
        )),
    color = alt.Color('Ano:O'),
    tooltip = ['sum(Quantidade de Crimes)', 'Ano']
).properties(
    width=800,   # Largura personalizada em pixels
    height=450,  # Altura personalizada em pixels
    title=alt.TitleParams(text='Crimes Reportados por hora do dia', fontSize=20)
).configure_view(stroke=None).add_params(
    selection
).transform_filter(
    selection
)

alt.Chart(...)

_Análises_:

_(para fazer a análise tem que levar em conta que o ano de 2023 ainda não acabou)_